In [64]:
%reload_ext autoreload
%autoreload 1
%aimport config

conf = config.Config()

from model import MultiTask
model = MultiTask.load_model('saved_models/test.pth')

from dataloader import MultiDataset, MultiDataLoader
import torch

labels = " 'ABCDEFGHIJKLMNOPQRSTUVWXYZ_"


dataset = MultiDataset('data/splits/dev.csv', labels, 
                       use_mfcc_in=model._meta['use_mfcc_in'], 
                       use_ivectors_in=True,#model._meta['use_ivectors_in'], 
                       use_embeddings_in=True,#model._meta['use_embeddings_in'],
                       use_transcripts_out=model._meta['use_transcripts_out'], 
                       use_accents_out=model._meta['use_accents_out'])

dataloader = MultiDataLoader(dataset, batch_size=20, shuffle=False)

for data in dataloader:
    print(data[0].size())
    break

torch.Size([20, 960, 240])


In [70]:
import torch
sum([torch.tensor([2]), torch.tensor([5])])

tensor([7])

In [65]:
t='this is test'
i = t.find(' ')

In [66]:
t[i+1:]

'is test'

In [67]:
conf.patch_config('experiments.cfg')

[{'exp_name_prefix': 'a',
  'epochs': 2,
  'labels': "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ ",
  'batch_size': 10,
  'num_workers': 4,
  'cuda': True,
  'losses_mix': 0.9,
  'learning_rate': 0.0003,
  'mfcc_size': 40,
  'ivector_size': 100,
  'embedding_size': 100,
  'rnn_type': torch.nn.modules.rnn.GRU,
  'rnn_hidden_size': 800,
  'nb_head_layers': 3,
  'nb_speech_layers': 1,
  'nb_accents_layers': 1,
  'bidirectional': True,
  'bottleneck_size': 256,
  'use_mfcc_in': True,
  'use_ivectors_in': True,
  'use_embeddings_in': True,
  'use_transcripts_out': True,
  'use_accents_out': False,
  'decoder_alpha': 0.8,
  'decoder_beta': 1.0,
  'decoder_cutoff_top_n': 40,
  'decoder_cutoff_prob': 1.0,
  'decoder_beam_width': 100,
  'lm_path': './data/language_models/cv.lm',
  'train_manifest': './data/splits/train.csv',
  'dev_manifest': './data/splits/dev.csv',
  'test_manifest': './data/splits/test.csv',
  'tensorboard_path': './tensorboard_runs/',
  'saved_models_path': './saved_models/'},
 {'exp_nam

In [4]:
from model import MultiTask

model = MultiTask(DEBUG=False, rnn_hidden_size=800, 
                  use_mfcc_in=conf['use_mfcc_in'], 
                  use_ivectors_in=conf['use_ivectors_in'], 
                  use_embeddings_in=conf['use_embeddings_in'],
                  use_transcripts_out=conf['use_transcripts_out'], 
                  use_accents_out=conf['use_accents_out'])

In [24]:
print('blib', '\n')
print('test')


blib 

test


In [25]:
dataset.accent_dict

{'australia': 0, 'canada': 1, 'england': 2, 'us': 3}

In [26]:
conf['use_embeddings_in']

[True]

In [27]:
model._meta['use_embeddings_in']

True

In [29]:
from tqdm import tqdm_notebook as tqdm

model = model.cuda()

for data in tqdm(dataloader):
    inputs, inputs_lens, transcripts, transcripts_lens, accents = data

    
    a, b, c, __ = model(inputs.cuda(), inputs_lens.cuda())

In [10]:
MultiTask.serialize(model, 'tmp')

modelb = MultiTask.load_model('tmp')


In [12]:
modelb = modelb.cuda()

In [13]:
for data in tqdm(dataloader):
    inputs, inputs_lens, transcripts, transcripts_lens, accents = data

    
    a, b, c = modelb(inputs.cuda(), inputs_lens.cuda())


In [ ]:
@classmethod
def load_model(cls, path):
    package = torch.load(path, map_location=lambda storage, loc: storage)
    model = cls(rnn_hidden_size=package['hidden_size'], nb_layers=package['nb_layers'],
                labels=package['labels'], audio_conf=package['audio_conf'],
                rnn_type=supported_rnns[package['rnn_type']], bidirectional=package.get('bidirectional', True))
    model.load_state_dict(package['state_dict'])
    for x in model.rnns:
        x.flatten_parameters()
    return model

@classmethod
def load_model_package(cls, package):
    model = cls(rnn_hidden_size=package['hidden_size'], nb_layers=package['nb_layers'],
                labels=package['labels'], audio_conf=package['audio_conf'],
                rnn_type=supported_rnns[package['rnn_type']], bidirectional=package.get('bidirectional', True))
    model.load_state_dict(package['state_dict'])
    return model

@staticmethod
def serialize(model, optimizer=None, epoch=None, iteration=None, loss_results=None,
              main_loss_results=None, side_loss_results=None,
              cer_results=None, wer_results=None, mca_results=None, avg_loss=None, meta=None):
    model = model.module if DeepSpeech.is_parallel(model) else model
    package = {
        'version': model._version,
        'hidden_size': model._hidden_size,
        'nb_layers': model._nb_layers,
        'rnn_type': supported_rnns_inv.get(model._rnn_type, model._rnn_type.__name__.lower()),
        'audio_conf': model._audio_conf,
        'labels': model._labels,
        'state_dict': model.state_dict(),
        'bidirectional': model._bidirectional
    }
    if optimizer is not None:
        package['optim_dict'] = optimizer.state_dict()
    if avg_loss is not None:
        package['avg_loss'] = avg_loss
    if epoch is not None:
        package['epoch'] = epoch + 1  # increment for readability
    if iteration is not None:
        package['iteration'] = iteration
    if loss_results is not None:
        package['loss_results'] = loss_results
        package['main_loss_results'] = main_loss_results
        package['side_loss_results'] = side_loss_results
        package['cer_results'] = cer_results
        package['wer_results'] = wer_results
        package['mca_results'] = mca_results
    if meta is not None:
        package['meta'] = meta
    return package